## Yolov5 related imports

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
'''#detecting for a video using given yolov5 detect.py file 
!python detect.py --weights /content/gdrive/MyDrive/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/hard-hat-pic'''

## Functions required for yolov5

In [5]:
#---------------------------------------
import torch.nn as nn
#Functions used for yoloV5
class Ensemble(nn.ModuleList):
    # Ensemble of models
    def __init__(self):
        super(Ensemble, self).__init__()

    def forward(self, x, augment=False):
        y = []
        for module in self:
            y.append(module(x, augment)[0])
        # y = torch.stack(y).max(0)[0]  # max ensemble
        # y = torch.stack(y).mean(0)  # mean ensemble
        y = torch.cat(y, 1)  # nms ensemble
        return y, None  # inference, train output

def attempt_load(weights, map_location=None):
    # Loads an ensemble of models weights=[a,b,c] or a single model weights=[a] or weights=a
    model = Ensemble()
    for w in weights if isinstance(weights, list) else [weights]:
        attempt_download(w)
        ckpt = torch.load(w, map_location=map_location)  # load
        model.append(ckpt['ema' if ckpt.get('ema') else 'model'].float().fuse().eval())  # FP32 model

    # Compatibility updates
    for m in model.modules():
        if type(m) in [nn.Hardswish, nn.LeakyReLU, nn.ReLU, nn.ReLU6, nn.SiLU]:
            m.inplace = True  # pytorch 1.7.0 compatibility
        elif type(m) is Conv:
            m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility

    if len(model) == 1:
        return model[-1]  # return model
    else:
        print('Ensemble created with %s\n' % weights)
        for k in ['names', 'stride']:
            setattr(model, k, getattr(model[-1], k))
        return model  # return ensemble

## Main cell block

In [ ]:
#imports
import torch
import torch.nn as nn
import time
import cv2 as cv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from models.common import Conv, DWConv
from utils.google_utils import attempt_download
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
import os
import matplotlib
import glob
import time
import argparse
import cv2
import numpy as np
import math 

#loading torch model
device =  "cpu"
weights = '/content/gdrive/MyDrive/best.pt'
model = attempt_load(weights, map_location=device)  
cap = cv2.VideoCapture('/content/gdrive/MyDrive/safety_video/Importance of wearing Safety helmets at work.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
result = cv2.VideoWriter('/content/gdrive/MyDrive/output_final.mp4', fourcc, 20.0, (frame_width,frame_height))
frame_count=0
import cv2

#main while loop
while True:
    print("Frame count = ",frame_count)
    frame_count+=1
    ret, img = cap.read()
    if not ret:
        break
    if img is not None:
      frame = img.copy()
      whT = 480
      frame = cv2.resize(frame , (whT , whT))
      frame = frame[:, :, ::-1].transpose(2, 0, 1)
      frame = np.ascontiguousarray(frame)
      height,width,layers=img.shape
      #---------------forwarding the torch model (yolo v5)--------
      frame = frame / 255.0
      torch_img = torch.from_numpy(frame).unsqueeze(0).to(device)
      with torch.no_grad():
          pred = model(torch_img.float() , augment = False)
          det_objects = non_max_suppression(pred[0], 0.6, 0.3 , None , False)[0][...,:5].numpy()
      print("detected objects",len(det_objects))
      objects = [[int((xmin/whT)*width) , int((ymin/whT)*height) , int((xmax/whT)*width) , int((ymax/whT)*height) , score]  for xmin , ymin , xmax , ymax , score in det_objects ]
      objects = [[int(xmin) , int(ymin) , int(xmax) , int(ymax) , score]  for xmin , ymin , xmax , ymax , score in objects]
      min_dist = np.inf
      obj = 0

      #-----------------
      if len(objects) == 0:
          #if no objects were detected
          print("No objects found in frame!")
          k = cv2.waitKey(33)
          result.write(img)
          #cv2_imshow(img)
          if k==27:    # Esc key to stop
              cv2.destroyAllWindows()
              break
          elif k==-1:  # normally -1 returned,so don't print it
              continue
      else:
          #if atleast 1 obbject is detected
          limit_objects = np.array(objects)
          for obj in limit_objects:
              xmin  , ymin , xmax , ymax , score = obj
              if xmin < 0 or ymin < 0 or xmax < 0 or ymax < 0:
                  continue
              xmin , ymin , xmax , ymax  = int(xmin) , int(ymin) , int(xmax) , int(ymax)
              img_object = img[int(ymin):int(ymax), int(xmin):int(xmax)] #cropping detected object
              crop_img=img_object[0:math.floor(img_object.shape[0]/3),0:math.floor(img_object.shape[1])] #getting top 1/3rd region of image ( our region of interest)
              crop_centre_y=crop_img.shape[0]//2
              crop_centre_x=crop_img.shape[1]//2
              crop_centre_coord=(crop_centre_x,crop_centre_y)#centre point of R.O.I 
              b,g,r=crop_img[crop_centre_y,crop_centre_x] #BGR values of centre point of R.O.I
              final_color=[int(b),int(g),int(r)]
              img=cv2.rectangle(img , (xmin , ymin) , (xmax , ymax) , final_color , 2) #displaying rectangle with color of hard hat
              result.write(img)
              
result.release()
cap.release()
cv2.destroyAllWindows()